In [140]:
import pandas as pd
diary_train = pd.read_csv('diary_train.csv')
diary_test = pd.read_csv('diary_test.csv')

In [141]:
from transformers import (
    AutoTokenizer,
    BartForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from tokenizers import Tokenizer
from typing import Dict, List, Optional
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
from typing import Dict

In [3]:
# HashTag Evaluation

In [204]:
from transformers import BartForConditionalGeneration
model_name = "jjae/kobart-hashtag"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model2 = BartForConditionalGeneration.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [168]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [205]:
model2 = model2.to(device)

In [206]:
def make_tag(text, label):
  # 입력 문장을 토큰화하여 인코딩
  input_ids = tokenizer.encode(text, return_tensors="pt").to(device)

  # 모델에 입력 전달하여 디코딩
  output = model2.generate(input_ids = input_ids, bos_token_id = model2.config.bos_token_id,
                          eos_token_id = model2.config.eos_token_id, length_penalty = 3.0, max_length = 50, num_beams = 4)

  # 디코딩된 출력을 토크나이저를 사용하여 텍스트로 변환
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

  #print("입력 문장:", text,'\n')
  #print("모델 출력:", decoded_output,'\n')
  #print("정답 레이블:", label)
  return decoded_output

In [207]:
diary_test.iloc[1,2]

'#자유로운시간활용 #여가활동 #독서 #음악감상 #산책'

In [208]:
make_tag(diary_test.iloc[1,1], diary_test.iloc[1,2])

'#자유로운시간활용 #여가활동 #독서 #음악감상 #산책'

In [209]:
make_tag(diary_test.iloc[-1,1], diary_test.iloc[-1,2])

'#아트마켓 #페스티벌참가 #창작물전시 #소통 #예술의힘'

In [210]:
make_tag(diary_test.iloc[7,1], diary_test.iloc[7,2])

'#출근길실수 #버스를 놓쳤다 #부주의와급함 #실패와상사의질책 #배움과개선'

In [211]:
make_tag(diary_test.iloc[23,1], diary_test.iloc[23,2])

'#공부계획 #스케줄관리 #목표설정 #꾸준한노력 #스트레스최소화'

In [212]:
# Test (New data)
txt = "가족과 함께 해변으로 소풍을 다녀왔어요. 바다 풍경을 감상하고 놀이를 즐기며 소중한 시간을 보내고 돌아오는 길에는 행복함과 만족감을 느꼈어요. 가족과 함께한 소중한 추억을 간직하게 되었어요."
lbl = '#가족 #해변소풍 #소중한시간 #행복한추억'
make_tag(txt, lbl)

'#가족과함께 #해변소풍 #소중한추억 #행복한시간 #가족사랑'

In [213]:
# 모델 output열 생성
diary_test['hash_tag_out'] = diary_test.apply(lambda x: make_tag(x[1], x[2]), axis = 1)

C:\Users\user\AppData\Local\Temp\ipykernel_7568\3364559632.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  diary_test['hash_tag_out'] = diary_test.apply(lambda x: make_tag(x[1], x[2]), axis = 1)


In [214]:
diary_test.head()

,diary,summary,hashtag,hash_tag_out
0,아침 일어나자마자 조용한 공간에서 명상을 시작했어. 깊은 숨을 들이마시며 마음을 가...,자기계발을 위해 명상과 여유를 추구한 날이었어요. 명상을 통해 마음의 여유와 집중력...,#자기계발 #명상과여유 #평화와안정 #내면의평화 #더나은사람되기,#자기계발 #명상과여유 #마음의여유 #평화와안정 #평화와안정
1,바쁜 일상에서 벗어나서 나만의 시간을 가지고 즐거운 여가 활동을 즐길 수 있어서 기...,"자유로운 시간을 활용하여 여가 활동에 몰두한 하루였어요. 독서, 음악 감상, 산책 ...",#자유로운시간활용 #여가활동 #독서 #음악감상 #산책,#자유로운시간활용 #여가활동 #독서 #음악감상 #산책
2,독립과 자립은 우리에게 큰 자유를 준다. 우리는 자유롭게 선택하고 행동할 수 있으며...,"독립과 자립을 통해 우리는 자유를 얻지만, 그에 따른 책임도 져야 한다. 자유로운 ...",#독립과자립 #자유와책임 #선택과책임 #자유로운선택 #균형있는삶,#독립과자립 #자유와책임 #상호보완적인 요소 #균형과균형 #자유와책임 #균형과균형
3,"나는 내가 가진 강점을 찾아보고, 그것을 발휘하며 성취를 이루어갈 수 있는 시간을 ...",자신의 강점을 인식하고 이를 통해 성취감을 느끼는 날이었다. 나의 분석력과 창의적인...,#자신의강점 #성취감 #분석력 #창의적사고 #자신감,#자점인식 #성취감 #분석력 #창의적사고 #아이디어발전
4,시험장을 나와서 자유로운 숨을 쉬며 가벼운 발걸음으로 친구들과 함께 밖으로 나왔어....,시험이 끝나고 해방감을 느끼며 친구들과 밖으로 나가서 자유롭게 즐거운 시간을 보냈어.,#시험이끝 #자유로운느낌 #친구와산책 #즐거운시간 #열심히공부하며즐기기,#시험끝 #해방감 #친구들과함께 #자유로운시간 #즐거움


In [194]:
diary_test.hash_tag_out.iloc[:30]

0                 #자기계발 #명상과여유 #마음의여유 #평화와안정 #평화와안정
1                     #자유로운시간활용 #여가활동 #독서 #음악감상 #산책
2     #독립과자립 #자유와책임 #상호보완적인 요소 #균형과균형 #자유와책임 #균형과균형
3                    #자점인식 #성취감 #분석력 #창의적사고 #아이디어발전
4                    #시험끝 #해방감 #친구들과함께 #자유로운시간 #즐거움
5                     #과학소품 #구매 #소품제작 #과학적사고 #창의력발휘
6                       #내적안정 #평화로움 #행복 #삶의만족도 #감사함
7          #출근길실수 #버스를 놓쳤다 #부주의와급함 #실패와상사의질책 #배움과개선
8                #주변환경소통 #소통과이해 #관계중요성 #가족과함께 #의견존중
9                        #가족여행 #풍경 #맛있는음식 #즐거움 #유대감
10                  #주변평가 #이해와존중 #자신감 #진정한평가 #진정한평가
11                   #복습 #자신감키우기 #시험준비 #긴장과기대 #잠에들다
12                  #자작곡 #가사쓰기 #감정표현 #음악으로소통 #소중한추억
13                      #가족변화 #적응과이해 #유대감 #서로지지 #배려
14                #지역사회 #다양성존중 #의견공유 #이해와존중 #지역사회참여
15                       #가족사진 #사진찍기 #편집 #창의적작업 #공유
16                  #코딩챌린지 #프로그래밍능력향상 #아이디어해결 #문제해결
17       #창의적인도전 #프로젝트시작 #시장진입과경쟁 #성공과사회적변화 #지속적인성공
18                    #새로운환경 #적응과변화 #성숙한인식 #다양성 #교류
19          

In [180]:
diary_test.hash_tag_out.iloc[30:60]

30                      #꿈 #유명인과만남 #협업 #성장과발전 #열정과노력
31                    #꿈과목표 #자기계발 #지속적인학습 #꾸준한노력 #배움
32                        #전공과일상균형 #시간부족 #의욕저하 #조언필요
33          #환경문제인식 #지속가능한삶 #작은노력큰변화 #건강한지구 #작은노력큰변화
34          #인간관계 #불안과고민극복 #자기반성 #소통과의견공유 #긍정적인사고와자세
35                 #갑작스러운날씨 #몸상태영향 #예민한반응 #대응력 #건강관리
36                          #가치관 #신념 #논의 #탐구 #관점 #인식
37                #새로운프로젝트 #업무완료 #동료협력 #성취와보람 #도전과성장
38                              #가족 #취미 #관심사 #이해 #존중
39              #음악앱 #음악스트리밍 #개인화된추천 #플레이리스트 #음악을즐기다
40                #스터디그룹 #친구들과공부 #강점발휘 #학습효율 #친구들과공부
41            #종강후복수전공 #졸업요건확인 #학업계획 #미래방향성 #책임감과성취감
42                    #구름 #나무 #자연의아름다움 #평온함 #자연의아름다움
43          #건강을유지하며 #삶의만족도 #운동과식습관 #스트레스해소 #자신감과만족감
44              #수수업성적향상목표 #친구와의이야기 #꾸준한복습 #문제해결 #소통
45    #숙소에서의휴식 #안락한분위기 #아늑한소파와 조용한분위기 #편안한침대 #안락한분위기
46                #친구들과함께 #순간과추억 #우정을깊게 #에너지와힘 #강한우정
47                        #자아실현 #창의적활동 #행복 #자기개발 #보람
48                  #가족과함께 #요리와베이킹 #맛있는음식 #협력 

In [275]:
diary_test.to_csv("diary_test_out.csv", index = False, encoding = 'utf-8', sep = ',')

In [182]:
# 해시태그 개수 확인
diary_test['hash_tag_out'].map(lambda x: x.count("#")).value_counts()

hash_tag_out
5    270
4     20
6     11
Name: count, dtype: int64

## Rouge

In [19]:
!pip install rouge


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [215]:
from rouge import Rouge
rouge_score = Rouge()

In [216]:
def cal_rouge(original, reference):
  original = "".join(original.split("#")).strip() # 해시태그 제거 후 측정
  reference = "".join(reference.split("#")).strip()
  score = rouge_score.get_scores([original], [reference], avg = True)
  return score

In [217]:
r1_r = r1_p = r1_f = 0
r2_r = r2_p = r2_f = 0
rl_r = rl_p = rl_f = 0

for i in range(len(diary_test)):
    scores = cal_rouge(diary_test.iloc[i,2], diary_test.iloc[i,3])
    r1_r += scores['rouge-1']['r']
    r1_p += scores['rouge-1']['p']
    r1_f += scores['rouge-1']['f']
    r2_r += scores['rouge-2']['r']
    r2_p += scores['rouge-2']['p']
    r2_f += scores['rouge-2']['f']
    rl_r += scores['rouge-l']['r']
    rl_p += scores['rouge-l']['p']
    rl_f += scores['rouge-l']['f']

r1_r = round(r1_r / len(diary_test), 3)
r1_p = round(r1_p / len(diary_test), 3)
r1_f = round(r1_f / len(diary_test), 3)
r2_r = round(r2_r / len(diary_test), 3)
r2_p = round(r2_p / len(diary_test), 3)
r2_f = round(r2_f / len(diary_test), 3)
rl_r = round(rl_r / len(diary_test), 3)
rl_p = round(rl_p / len(diary_test), 3)
rl_f = round(rl_f / len(diary_test), 3)

In [218]:
# ROUGE (average) - 해시태그이므로 rouge-2부터는 의미 없음
tag_rouge = dict({'rouge-1': dict({'r': r1_r, 'p': r1_p, 'f': r1_f}),
      'rouge-2': dict({'r': r2_r, 'p': r2_p, 'f': r2_f}),
      'rouge-l': dict({'r': r1_r, 'p': r1_p, 'f': r1_f})})

In [219]:
tag_rouge

{'rouge-1': {'r': 0.523, 'p': 0.481, 'f': 0.497},
 'rouge-2': {'r': 0.279, 'p': 0.264, 'f': 0.269},
 'rouge-l': {'r': 0.523, 'p': 0.481, 'f': 0.497}}

## BertScore

In [220]:
!pip install bert_score


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [221]:
import bert_score

def bertScore(original, reference):
  original = " ".join(original.split("#")) # 해시태그 제거 후 측정
  reference = " ".join(reference.split("#"))
  orig = [original]
  ref = [reference]
  P,R,F = bert_score.score(orig, ref, lang='ko')
  return P, R, F

In [222]:
P = R = F = 0

for i in range(len(diary_test)):
    p, r, f = bertScore(diary_test.iloc[i,2], diary_test.iloc[i,3])
    P+=p
    R+=r
    F+=f

R = round(R.item() / len(diary_test), 3)
P = round(P.item() / len(diary_test), 3)
F = round(F.item() / len(diary_test), 3)

In [223]:
# BertScore (average)
tag_bert = dict({'recall': R, 'precision': P, 'f1-score': F})

In [224]:
tag_bert

{'recall': 0.877, 'precision': 0.867, 'f1-score': 0.872}

In [225]:
# Summary Evaluation

In [253]:
from transformers import BartForConditionalGeneration
model_name = "jjae/summary"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [254]:
def diary_summary(text):
  # Encode input text
  input_ids = tokenizer.encode(text, return_tensors = 'pt').to(device)
  # Generate summary text ids
  summary_text_ids = model.generate(input_ids = input_ids,
                                    bos_token_id = model.config.bos_token_id,
                                    eos_token_id = model.config.eos_token_id,
                                    length_penalty = 2.0,
                                    max_length = 150,
                                    num_beams = 2)
  return tokenizer.decode(summary_text_ids[0], skip_special_tokens = True)

In [255]:
diary_summary(diary_test.iloc[20,0])

"봄날 '봄날'이라는 노래에 대한 생각을 통해 감사함과 행복을 느꼈어요."

In [231]:
diary_summary(diary_test.iloc[27,0])

'좋아하는 음악을 들으며 스트레스를 풀 수 있는 시간을 가졌어. 음악은 나에게 큰 위로와 힘을 주는 존재야.'

In [256]:
# 모델 output열 생성
diary_test['summary_out'] = diary_test.apply(lambda x: diary_summary(x[0]), axis = 1)

C:\Users\user\AppData\Local\Temp\ipykernel_7568\1527206841.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  diary_test['summary_out'] = diary_test.apply(lambda x: diary_summary(x[0]), axis = 1)


In [257]:
diary_test

,diary,summary,hashtag,hash_tag_out,summary_out
0,아침 일어나자마자 조용한 공간에서 명상을 시작했어. 깊은 숨을 들이마시며 마음을 가...,자기계발을 위해 명상과 여유를 추구한 날이었어요. 명상을 통해 마음의 여유와 집중력...,#자기계발 #명상과여유 #평화와안정 #내면의평화 #더나은사람되기,#자기계발 #명상과여유 #마음의여유 #평화와안정 #평화와안정,"명상과 여유를 통해 내 안의 평화와 안정을 찾고, 자기계발에 노력할 것을 다짐했어요."
1,바쁜 일상에서 벗어나서 나만의 시간을 가지고 즐거운 여가 활동을 즐길 수 있어서 기...,"자유로운 시간을 활용하여 여가 활동에 몰두한 하루였어요. 독서, 음악 감상, 산책 ...",#자유로운시간활용 #여가활동 #독서 #음악감상 #산책,#자유로운시간활용 #여가활동 #독서 #음악감상 #산책,"자유로운 시간을 활용하여 여가 활동을 즐긴 날이었어요. 독서와 산책, 음악 감성을 ..."
2,독립과 자립은 우리에게 큰 자유를 준다. 우리는 자유롭게 선택하고 행동할 수 있으며...,"독립과 자립을 통해 우리는 자유를 얻지만, 그에 따른 책임도 져야 한다. 자유로운 ...",#독립과자립 #자유와책임 #선택과책임 #자유로운선택 #균형있는삶,#독립과자립 #자유와책임 #상호보완적인 요소 #균형과균형 #자유와책임 #균형과균형,독립과 자립을 통해 얻은 자유와 책임의 관계에 대해 고민하고 생각해보았다. 우리는 ...
3,"나는 내가 가진 강점을 찾아보고, 그것을 발휘하며 성취를 이루어갈 수 있는 시간을 ...",자신의 강점을 인식하고 이를 통해 성취감을 느끼는 날이었다. 나의 분석력과 창의적인...,#자신의강점 #성취감 #분석력 #창의적사고 #자신감,#자점인식 #성취감 #분석력 #창의적사고 #아이디어발전,자신의 강점을 인식하고 이를 통해 성취감을 느끼는 날이었다. 내가 가진 강점을 활용...
4,시험장을 나와서 자유로운 숨을 쉬며 가벼운 발걸음으로 친구들과 함께 밖으로 나왔어....,시험이 끝나고 해방감을 느끼며 친구들과 밖으로 나가서 자유롭게 즐거운 시간을 보냈어.,#시험이끝 #자유로운느낌 #친구와산책 #즐거운시간 #열심히공부하며즐기기,#시험끝 #해방감 #친구들과함께 #자유로운시간 #즐거움,시험장에서에서 친구들과 함께 산책하고 웃을 수 있는 시간을 보냈어. 시험 기간 동안...
...,...,...,...,...,...
296,우리는 함께 모여서 서로의 성과와 성공을 축하하며 기뻐함을 나눴어. 각자가 이룬 목...,친구들과의 서로의 성취와 성공을 축하하며 함께 성장하는 시간은 우정을 깊게 만들어주...,#성취와성공 #축하와응원 #강한우정 #자랑스러운순간 #소중한시간,#성취와성공 #축하와응원 #강한우정 #소중한시간 #응원과지지,친구들과의 서로의 성취와 성공을 축하하며 응원하는 시간은 우정을 깊게 만들어주고 응...
297,다른 사람들에게 도움이 되고자 하는 마음으로 나와 함께하는 이들과 함께 길거리에서 ...,"길거리 나눔 활동에 참여하여 음식과 생필품을 나누어주고, 도움이 필요한 사람들과 대...",#길거리나눔 #도움 #사회적연대감 #소통 #나눔의가치,#길거리나눔 #도움 #사회적연대 #소통 #사회적연대,길거리 나눔 활동에 참여하여 도움을 주는 경험을 했어요. 작지만 의미 있는 일로 사...
298,"삶은 끊임없는 변화와 도전을 안겨주는데, 나는 이런 변화에 어떻게 적응해야 할지 항...",인생의 변화와 그에 대한 적응에 대해 생각하게 된 하루였어요. 변화에 대한 두려움을...,#인생의변화 #적응력 #도전과성장 #새로운기회 #성공을향해,#인생의변화 #적응력 #성장과발전 #도전과성장 #새로운도전,인생의 변화에 대한 적응에 대해 고민하고 생각해보았어요. 변화에 대한 두려움을 떨쳐...
299,과학 잡지는 최신 연구 동향과 흥미로운 과학적인 이야기를 제공해주어 지식의 확장과 ...,과학 잡지를 읽는 시간을 가졌습니다. 최신 연구 동향과 흥미로운 이야기를 통해 과학...,#과학잡지 #지식확장 #과학연구 #우주탐사 #과학의매력 #학습,#과학잡지 # 최신연구동향 #흥미로운이야기 #과학지식 #지식확장,과학 잡지를 읽는 시간을 가졌습니다. 최신 연구 동향과 흥미로운 이야기를 통해 과학...


In [258]:
diary_test.iloc[3,0]

'나는 내가 가진 강점을 찾아보고, 그것을 발휘하며 성취를 이루어갈 수 있는 시간을 가졌다. 아침에 일어나서부터 나는 내가 가진 강점에 대해 생각했다. 나는 분석력이 뛰어나고 창의적인 사고를 가지고 있다는 것을 알았다. 그래서 오늘은 이 강점들을 활용하여 문제를 해결하고 아이디어를 발전시킬 것이다. 일을 시작하기 전에 나는 목표를 설정했다. 나의 강점을 최대한 활용하여 어떤 문제를 해결하고 어떤 목표를 달성할 것인지를 명확히 정했다. 목표가 뚜렷해지면서 나에게 동기와 열정이 생겼고, 성취를 이루기 위한 열쇠가 되었다. 작업을 진행하면서 나는 내 강점을 살리기 위해 노력했다. 분석력을 발휘하여 문제를 해결하고, 창의적인 사고를 통해 새로운 아이디어를 도출했다. 이 과정에서 나는 자신의 강점을 믿고 발휘함으로써 성취감을 느낄 수 있었다. 작업을 마친 후에는 나의 성취를 돌아보며 기쁨과 만족감을 느꼈다. 내가 가진 강점을 적극적으로 활용하여 뚜렷한 결과를 이뤄냈다는 것에 자신감을 얻었다. 성취의 순간들이 나를 더욱 발전시키고, 더 큰 도전과 성공을 향한 동기부여를 제공해주었다. 하루가 저물어가는 시간에는 나 자신에게 칭찬과 격려의 말을 건네기로 했다. 내가 가진 강점을 인식하고 그것을 발휘함으로써 어떤 성취를 이루었는지를 돌아보며 나 자신에게 감사하고 칭찬해주는 시간을 가졌다. 오늘의 경험을 돌아보며, 자신의 강점을 인식하고 이를 통해 성취감을 느끼는 일은 나에게 큰 힘이 되고 있다는 것을 깨달았다. 이런 소중한 경험을 통해 나는 더욱 자신감을 키우고, 더 큰 도전과 성취를 이루어나갈 수 있을 것이다.'

In [259]:
diary_test.iloc[20,-1]

"봄날 '봄날'이라는 노래에 대한 생각을 통해 감사함과 행복을 느꼈어요."

In [260]:
diary_test.to_csv("diary_test_out.csv", index = False)

In [261]:
# 모델이 생성한 요약 글자수 확인
diary_test['summary_out'].map(lambda x: len(x)).describe()

count    301.000000
mean      93.159468
std       35.016298
min       22.000000
25%       68.000000
50%       88.000000
75%      112.000000
max      208.000000
Name: summary_out, dtype: float64

In [262]:
diary_test['summary_out'].map(lambda x: len(x)).sort_values()

86      22
74      31
11      32
189     35
75      36
      ... 
24     202
79     202
164    207
12     207
137    208
Name: summary_out, Length: 301, dtype: int64

## Rouge

In [263]:
def cal_rouge(original, reference):
  score = rouge_score.get_scores([original], [reference], avg = True)
  return score

In [264]:
r1_r = r1_p = r1_f = 0
r2_r = r2_p = r2_f = 0
rl_r = rl_p = rl_f = 0

for i in range(len(diary_test)):
    scores = cal_rouge(diary_test.iloc[i,1], diary_test.iloc[i,-1])
    r1_r += scores['rouge-1']['r']
    r1_p += scores['rouge-1']['p']
    r1_f += scores['rouge-1']['f']
    r2_r += scores['rouge-2']['r']
    r2_p += scores['rouge-2']['p']
    r2_f += scores['rouge-2']['f']
    rl_r += scores['rouge-l']['r']
    rl_p += scores['rouge-l']['p']
    rl_f += scores['rouge-l']['f']

r1_r = round(r1_r / len(diary_test), 3)
r1_p = round(r1_p / len(diary_test), 3)
r1_f = round(r1_f / len(diary_test), 3)
r2_r = round(r2_r / len(diary_test), 3)
r2_p = round(r2_p / len(diary_test), 3)
r2_f = round(r2_f / len(diary_test), 3)
rl_r = round(rl_r / len(diary_test), 3)
rl_p = round(rl_p / len(diary_test), 3)
rl_f = round(rl_f / len(diary_test), 3)

In [265]:
# ROUGE (average)
tag_rouge = dict({'rouge-1': dict({'r': r1_r, 'p': r1_p, 'f': r1_f}),
      'rouge-2': dict({'r': r2_r, 'p': r2_p, 'f': r2_f}),
      'rouge-l': dict({'r': r1_r, 'p': r1_p, 'f': r1_f})})

In [266]:
tag_rouge

{'rouge-1': {'r': 0.559, 'p': 0.488, 'f': 0.511},
 'rouge-2': {'r': 0.41, 'p': 0.359, 'f': 0.375},
 'rouge-l': {'r': 0.559, 'p': 0.488, 'f': 0.511}}

## BertScore

In [267]:
import bert_score

def bertScore(original, reference):
  orig = [original]
  ref = [reference]
  P,R,F = bert_score.score(orig, ref, lang='ko')
  return P, R, F

In [268]:
P = R = F = 0

for i in range(len(diary_test)):
    p, r, f = bertScore(diary_test.iloc[i,2], diary_test.iloc[i,3])
    P+=p
    R+=r
    F+=f

R = round(R.item() / len(diary_test), 3)
P = round(P.item() / len(diary_test), 3)
F = round(F.item() / len(diary_test), 3)

In [269]:
# BertScore (average)
summary_bert = dict({'recall': R, 'precision': P, 'f1-score': F})

In [270]:
summary_bert

{'recall': 0.899, 'precision': 0.892, 'f1-score': 0.895}

In [271]:
diary_test.iloc[-1,-1]

'아트 마켓이나 페스티벌에 참가하여 작품을 감상하고 다른 작가들의 작품을 감상하는 경험을 했어요. 예술에 대한 관점과 경험을 넓힐 수 있는 경험이었어요.'

In [272]:
diary_test.iloc[-1,1]

'아트 마켓이나 페스티벌에 참가하여 창작물을 전시하고 다른 사람들과 소통하는 경험을 했어요. 작품에 대한 이야기를 나누고, 다른 작가들의 작품을 감상하며 예술의 힘과 소통의 가치를 깨달았어요.'

In [274]:
diary_test.iloc[7,-1]

'출근길에 버스를 놓치고 말았는데, 실패와 상사의 질책을 통해 자신의 부주의한 태도와 급함으로 인한 실수를 깨달았어요. 이를 통해 자신의 부족한 점을 인정하고 개선해야 한다는 교훈을 얻었어요.'